In [1]:
import numpy as np
import cv2
from skimage import color, segmentation, filters
from sklearn.metrics import jaccard_score  # Using for IoU

In [2]:
# Calculate IoU score
def iou_score(mask_true, mask_pred):
    mask_true_flat = mask_true.flatten()
    mask_pred_flat = mask_pred.flatten()
    return jaccard_score(mask_true_flat, mask_pred_flat, average='binary')

In [3]:
# Process video with active contour and region growing
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Unable to open video")
        exit()

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 60
    out = cv2.VideoWriter('result (library).mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_smooth = filters.gaussian(color.rgb2gray(frame_rgb), sigma=1)

    # Initial snake contour (circular in the center of the image)
    h, w = gray_smooth.shape
    s = np.linspace(0, 2 * np.pi, 200)
    r, c = h // 2 + 100 * np.sin(s), w // 2 + 100 * np.cos(s)
    initial_contour = np.array([r, c]).T

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray_smooth = filters.gaussian(color.rgb2gray(frame_rgb), sigma=1)

        # Use skimage's active_contour to evolve the snake in the current frame
        snake = segmentation.active_contour(gray_smooth, initial_contour, alpha=0.1, beta=10, gamma=0.1, max_num_iter=100)

        # Create mask for the contour
        mask_contour = np.zeros_like(gray_smooth, dtype=np.uint8)
        cv2.polylines(mask_contour, [np.int32(snake)], isClosed=True, color=1, thickness=1)

        # Display histograms and determine the average intensity across channels
        avg_intensity = np.mean(frame_rgb[:, :, np.argmin([np.mean(frame_rgb[:, :, i]) for i in range(3)])])
        threshold = avg_intensity

        # Apply region growing based on the average intensity
        image_channel = frame_rgb[:, :, np.argmin([np.mean(frame_rgb[:, :, i]) for i in range(3)])].astype(np.uint8)
        mask = np.zeros((image_channel.shape[0] + 2, image_channel.shape[1] + 2), np.uint8)
        seed_point = (frame_rgb.shape[1] // 2, frame_rgb.shape[0] // 2)
        cv2.floodFill(image_channel, mask, seed_point, 255, (threshold,) * 3, (threshold,) * 3, cv2.FLOODFILL_MASK_ONLY)
        region_mask = mask[1:-1, 1:-1]

        # IoU calculation between active contour and region-growing masks
        iou = iou_score(mask_contour > 0, region_mask > 0)
        print(f"IoU score for frame: {iou:.4f}")

        # Combine contour with original frame
        frame_with_contour = frame.copy()
        for point in snake:
            x, y = int(point[1]), int(point[0])
            cv2.circle(frame_with_contour, (x, y), 2, (0, 255, 0), -1)

        # Draw the region-growing result
        frame_with_contour[region_mask == 255] = (255, 0, 0)  # Color the region in red

        out.write(frame_with_contour)
        cv2.imshow('Contour Video', frame_with_contour)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [4]:
# Run the video processing
video_path = 'test.mp4'
process_video(video_path)

IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0025
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: 0.0024
IoU score for frame: